# Análise de reviews de produtos usando NLP

O presente trabalho pretende analisar os textos de revisão de produtos da base de dados obtida na plataforma Kaggle. Para isso serão usadas bibliotecas Python de processamento de linguagem natural já conhecidas como Spacy, Sci-kit Learn e.

https://www.kaggle.com/olistbr/brazilian-ecommerce

Objetivos:
- Encontrar caracaterísticas que distinguam textos de reviews com notas altas e com notas baixas.
- Analisar textos com análise de sentimentos e identificar textos muito negativos, mas que possuam notas altas.

In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import spacy
from nltk import corpus
import nltk
import re
from unicodedata import normalize

In [25]:
e_set = pd.read_csv("./brazilian-ecommerce/olist_public_dataset_v2.csv")
e_set.head()

,order_id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,...,product_name_lenght,product_description_lenght,product_photos_qty,product_id,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,b95df3cef5297e79ef709ba256518f6f,delivered,349.90,13.84,1,1,2017-01-31 17:19:01.000000,2017-02-01 02:41:21.549551,2017-03-15 00:00:00.000000,2017-02-06 11:04:24.154259,...,51,625,1,6cdd53843498f92890544667809f1595,b95df3cef5297e79ef709ba256518f6f,5,NaN,NaN,2017-02-07 00:00:00.000000,2017-02-09 02:37:37+00:00
1,59af46052a799e80e2f0c665c587731d,delivered,15.00,15.10,1,1,2017-09-09 19:52:54.000000,2017-09-10 20:03:31.535281,2017-10-02 00:00:00.000000,2017-09-13 20:17:41.296915,...,44,1428,2,ae5cad88462eb7b7b61401e31c45618e,59af46052a799e80e2f0c665c587731d,5,NaN,entrega em 2 dias produto c boa qualidade otim...,2017-09-14 00:00:00.000000,2017-09-15 03:43:47+00:00
2,a3e6136894621db402a772c6bc72a12a,delivered,238.90,18.00,1,1,2017-01-30 17:00:09.000000,2017-01-30 17:31:25.438253,2017-03-07 00:00:00.000000,2017-02-06 15:43:04.758566,...,55,637,1,0c9ff9d8ed9b9bdd825487b3a66e05f5,a3e6136894621db402a772c6bc72a12a,5,NaN,produto veio antes do prazo informado muito ob...,2017-02-07 00:00:00.000000,2017-02-10 14:18:53+00:00
3,b675ea5a618922f6e679e30531b8957b,delivered,29.99,18.23,1,1,2018-03-11 18:18:36.000000,2018-03-11 18:30:37.931962,2018-04-03 00:00:00.000000,2018-04-03 20:36:43.778451,...,55,617,1,ad0a798e7941f3a5a2fb8139cb62ad78,b675ea5a618922f6e679e30531b8957b,4,NaN,NaN,2018-04-04 00:00:00.000000,2018-04-05 02:52:31+00:00
4,195a8be6794c487fe6cfbb97b7c61902,delivered,295.99,47.65,1,1,2017-04-20 08:01:08.000000,2017-04-25 08:05:40.405383,2017-05-24 00:00:00.000000,2017-05-04 18:47:45.721758,...,49,558,2,eaf2046d4c87809247a30050ea13df03,195a8be6794c487fe6cfbb97b7c61902,5,NaN,NaN,2017-05-05 00:00:00.000000,2017-05-08 15:20:18+00:00


# Padronizando o DataFrame

Retiramos todas as linhas que não possuem reviews para analisar somente os textos.

In [44]:
e_work_set = e_set[['order_products_value','review_score', 'review_comment_message']].copy()
e_work_set = e_work_set.dropna().reset_index(drop = True)
e_work_set.head()

,order_products_value,review_score,review_comment_message
0,15.0,5,entrega em 2 dias produto c boa qualidade otim...
1,238.9,5,produto veio antes do prazo informado muito ob...
2,160.0,4,chegou bem antes do prazo ótimo vendedor
3,25.0,3,gostei do produto
4,187.0,5,o produto veio certinho muito bem embalado e b...


## Pre-processamento

## Remoção de acentuação 

In [156]:
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

### Remoção de Stopwords

As stopwords, palavras que não possuem sentido significativo para a sentença são retiradas através da corpora disponibilizada pela bibliteca NLTK.

In [157]:
stop_words = set(corpus.stopwords.words('portuguese'))

stop_words_norm = []
for i in stop_words:
    word = remover_acentos(i)
    stop_words_norm.append(word)
    
stop_words_norm = stop_words_norm.remove('nao')

## Lematização

In [109]:
def lematizar(lista_palavras):
    nlp = pt_core_news_sm.load()
    doc = nlp(' '.join(lista_palavras))
    lista_lemas = [w.lemma_ for w in doc]
    return lista_lemas

## Tokenização 

Agora fazemos a tokenização do texto e retornamos uma lista com os tokens. Fazemos isso para todos os textos do DataFrame.

In [118]:
for index,row in e_work_set.iterrows():
    tk_reviews = remover_acentos(e_work_set['review_comment_message'][index]).split()
    tk_reviews = lematizar(tk_reviews)
    e_work_set.loc[index,'review_tratada'] = ' '.join([w for w in tk_reviews if not w in stop_words_norm])
e_work_set.head()

,order_products_value,review_score,review_comment_message,review_tratada
0,15.0,5,entrega em 2 dias produto c boa qualidade otim...,entregar 2 dia produto c bom qualidade otimo c...
1,238.9,5,produto veio antes do prazo informado muito ob...,produto vir antar prazo informar obrigar pelar...
2,160.0,4,chegou bem antes do prazo ótimo vendedor,chegar bem antar prazo otimo vendedor
3,25.0,3,gostei do produto,gostar produto
4,187.0,5,o produto veio certinho muito bem embalado e b...,produto vir certo bem embalar bem antar prazo ...


In [119]:
e_work_set.to_csv('review_lematizada.csv',encoding = 'utf-8',index = False)

# Extraindo Features das reviews tokenizadas

Agora vamos extrair algumas características das reviews para analisarmos separadamente os textos de reviews para cada pontuação.

In [120]:
e_set_rev_1 = e_work_set[e_work_set['review_score'] == 1].reset_index(drop = True)
e_set_rev_2 = e_work_set[e_work_set['review_score'] == 2].reset_index(drop = True)
e_set_rev_3 = e_work_set[e_work_set['review_score'] == 3].reset_index(drop = True)
e_set_rev_4 = e_work_set[e_work_set['review_score'] == 4].reset_index(drop = True)
e_set_rev_5 = e_work_set[e_work_set['review_score'] == 5].reset_index(drop = True)
e_set_rev_1.head()

,order_products_value,review_score,review_comment_message,review_tratada
0,134.90,1,o correio ainda não fez a entrega to furioso,correar ainda fazer entregar to furioso
1,329.99,1,não recebi o meu produto e nem fui avisada da ...,receber produto ser avisar falto item loja ent...
2,251.90,1,comprei um produto e recebi outro e inferior g...,comprar produto receber outro inferior gostar ...
3,979.00,1,não comprem só vai dar dor de cabeça o produto...,comprar ir dar dor cabeca produto nunca chegar
4,62.70,1,comprei 2 produtos iguais porém em cores difer...,comprar 2 produto igual pôr corar diferente ch...


## Análise Exploratória

Será mostrado, como exemplo, o código para extração de cada feature nas reviews de nota 1. Em seguida mostraremos apenas os resultados obtidos para as reviews agrupadas por nota.

### Media de Tokens por texto:

In [31]:
def media_tokens(df):
    vectorizer = CountVectorizer(encoding = 'utf_8')
    reviews = []

    for row in df.itertuples():
        reviews.append(row[3])

    tokenized_reviews_all = vectorizer.fit_transform(reviews)
    nr_medio_de_tokens_1 = tokenized_reviews_all.toarray().sum()/len(e_set_rev_1)
    print(("%.2f" % nr_medio_de_tokens_1))


### Análise de Sentimento

A análise de sentimento proposta é feita de maneira bem simples. Baseado em uma lista de termos positivos e uma lista de termos negativos calculamos a porcentagem de palavras positivas e negativas no total. Palavras que não estão classificadas em positivas e negativas serão consideradas neutras.

As listas são provenientes do site Kaggle.
https://www.kaggle.com/rtatman/sentiment-lexicons-for-81-languages

### Importando léxico de palavras negativas

In [158]:
palavras_negativas = []

with open('./sentiment-lexicons/negative_words_pt.txt','r',encoding = 'utf_8') as f:
    for line in f.readlines():
        palavras_negativas.append(line)


lista_negativas = []
for i in palavras_negativas:
    palavra = i.replace('\n', '')
    lista_negativas.append(remover_acentos(palavra))   

lista_negativas = set(lista_negativas)


### Importando léxico de palavras positivas

In [159]:
palavras_positivas = []

with open('./sentiment-lexicons/positive_words_pt.txt','r',encoding = 'utf_8') as f:
    for line in f.readlines():
        palavras_positivas.append(line)


lista_positivas = []
for i in palavras_positivas:
    palavra = i.replace('\n', '')
    lista_positivas.append(remover_acentos(palavra))   

lista_positivas = set(lista_positivas)

### Definindo função para contagem de tokens positivos e negativos

In [160]:
# Recebe uma string e comparar as palavras que estão no léxico positivo e negativo

def sent_count(text):
    pos_count = 0
    neg_count = 0
    for token in text.split():
        if token in lista_positivas:
            pos_count += 1
        if token in lista_negativas:
            neg_count += 1
        return((pos_count,neg_count))


In [161]:
# Exemplo para as reviews com 1 estrela

for index,row in e_set_rev_1.iterrows():
    x = e_set_rev_1['review_tratada'][index]
    if x == '':
        continue
    sentimentos = sent_count(x)
    e_set_rev_1.loc[index,'nr_tk_positivos'] = sentimentos[0]
    e_set_rev_1.loc[index,'porcent_tk_positivos'] = sentimentos[0]/len(x.split())
    e_set_rev_1.loc[index,'nr_tk_negativos'] = sentimentos[1]
    e_set_rev_1.loc[index,'porcent_tk_negativos'] = sentimentos[1]/len(x.split())



In [140]:
x = e_set_rev_1.loc[e_set_rev_1['nr_tk_positivos'] != 0]
for index,row in x.iterrows():
    print(x['review_comment_message'][index],"\n")

não foi entregue o produto que comprei enviaram outro produto e solicitei a troca enviaram um voucher que não foi aceito pelo correios tive que ligar novamente e reclamar voltar no correios 

bom dia o lustre anunciado era azul claro foi entregue azul escuro 

foi entregue produto diferente do que comprei e estou encontrando dificuldades em devolvê lo 

não recomendo essa loja pra ninguém comprei o presente do meu filho no dia 23 de janeiro vai fazer um mês e nada ainda estão dizendo que o carteiro chamou e ninguém atendeu fiquei o dia inteiro emcasa 

não gostei do produto não funciona como aparece na propaganda 

valor do produto no site 175 00 mais frete de 12 73 emitiram nota fiscal no valor de 137 00 inferior ao valor do site por que 

não recomendo demorou muito para a entrega e sem contar que o produto é de péssima qualidade não veio completo faltou os parafusos e na hora da montagem a tinta saiu tudo extremamente chateada 

gostaria de lembrar que esse pedido eu tinha cancelado

não recomendo pois comprei o produto e ainda não chegou e já fiz vários contatos e nem se quer me deram um retorno decepcionada 

não foi entregue 

boa tarde não recebi meu produto pois cancelei o pedido pelo simples fato de ter tido um imprevisto na entrega janaína 

não recomendado 

eu recomendo a entrega chegou antes do prazo gostei muito 

darei 5 estrela e recomendaria a loja se o produto chegar em menos de um mês 

eu recomendaria pq o atraso da entrega deve ser pelo fato de ser final de ano mas ei queria q chagace pelo o menos o q eu paguei 

não foi entregue conforme solicitei fiz o pedido de 3 malas e recebemos apenas 2 tentei entrar em contato por e mail e por telefone e não obtive resposta 

não recomendo comprar nessa loja baratheon e ultimamente nas lannister os produtos demoram muito e as vezes é extraviado e eles não dão justificativa esse produto chegou no ultimo dia do prazo ruim 

gostei esperava que fosse melhor 

boa tarde recebi o tapete de eva e não recebi o bon

gostaria de cancelar a compra me deram o prazo de 15 dias para entrega comprei em 08 12 entendi o final de ano e aceitei que seria entregue dia 05 01 mas até agora nada muito instatisfeita 

gostaria de saber mais informações estava precisando do meu produto e nada ainda geralmente recebo antes do prazo mt antes 

precisa 30 dias pra fazer entrega do produto quando encerra o prazo de entrega apenas constataram que não havia o produto em estoque que falta de consideração com seus clientes 

não gostei do produto não centrifuga não indico 

boa noite estou aguardando retorno da loja nao recebi o pruduto 

boa tarde recebi apenas uma unidade do produto comprado por favor verificar o que houve aguardo 

simplesmente não entrega o produto 

so foi entregue 1 e foi comprado 2 

não recomendo esse produto pois o acabamento do produto não é bom a parte do alumínio é arranhado quero devolver o produto pelo site no prazo mas o site não está me dando a opção de devolução 

foi horrível a minha ex

foi entregue um produto que não foi pedido o que dá a entender que a empresa não está focada em atender bem seus clientes e nem satisfazer o que eles precisam quero devolução imediata do meu dinheiro 

bom dia gostaria de saber como que vai ficar como eu ainda não recebi o meu produto 

leva um tempo excessivo para enntregar um produto tão pequeno a compra é do dia 25 04 e até hoje não recebi a capa do celular um absurdo 

não gostei da qualidade do enchimento das almofadas 

realizei a compra de três cabos e recebi somente um porém paguei o preço dos três quero devolver o produto e receber meu dinheiro de volta 

não sei o que aconteceu estou tentando entender comprei o produto não recebi mandei mensagem não retornaram minha mensagem sempre compro no site lannister e nunca tive problema com entrega 

foi entregue um dos produtos o outro não foi entregue 

foram compradas duas cadeiras uma azul e outra verde foi entregue apenas a cadeira verde 

não sei quem errou só sei que fiquei fru

bom dia ainda não recebi meu produto dentro do prazo que disseram que entregariam estou decepcionado ainda estou aguardando o produto 

boa tarde não recebi um dos produtos que comprei só recebi um produto preciso que o outro seja entregue com urgência 

não recomendo essa loja à ninguém pois já fiz o pagamento da primeira parcela e até agora não recebi o produto 

não funciona pedi devolução 

boa tarde entregaram o pedido pela metade esta faltando a capa verde para o moto g 4 

só foi entregue a metade dos itens 

não recomendo este produto aparentemente falsificado o som é de péssima qualidade nada parecido com o original que eu tenho 

nao 

gostaria que entrassem em contato comigo pois comprei dois perfumes e só recebi um perfume 

foram dois produtos comprados e só mandaram um falta manda o outro 

bom 

sinceramente imagens meramente ilustrativas é um termo muito acurato para este produto estou tremendamente insatisfeito 

só foi entregue um dos produtos 

não deu dois dias a pe

...

não sabia que a lannister vendia produtos chineses de baixa qualidade se soubesse que era um réplica muito ruim por sinal teria comprado no camelô na esquina de casa pelo menos teria pago 1 3 do va 

eram 2 itens no mesmo pedido eles separam não tocaram a campainha e alegando não ter ninguém deixaram na agência dos correios me dando trabalho em ter que ir buscar 

foi entregue parcial faltam os 3 lustres 

deu problema na entrega e até agora nada 

não recomendo a loja baratheon demora na entrega e dificuldade com os correios estou acostumada a comprar no targaryen receber em casa meus pedidos no caso desta loja baratheon tive que ir retirar péssimo 

realizei o pedido do produto e após pagamento recebi a informação de que não tinha disponível em estoque 

realizei a compra de 3 unidades mas qual foi minha surpresa ao receber a caixa havia apenas uma unidade e a nota em anexo constavam as 3 de imediato entrei em contato com os correios me informaram q 

não gostei do produto compro fot

por favor eu já registrei há 48 horas atrás que não recebi a capa para colchão anti ácaro quemm da allergic center feita neste pedido recebi apenas as capas de travesseiro obs tel 9 8487 0788 

foi recebido apenas a pulseira de aço o kit com as duas pulseiras não veio o correio informou q só veio uma caixa com uma pulseira 

bom logo que eu observei demorando logo resolvi entra em contato com vocês a resposta foi espere até a data data que data se ja estava chegando e até o dia 18 12 17 nao tinha nem chegado em minha re 

não recomendo 

foram compradas duas cadeiras uma azul e outra verde foi entregue apenas a cadeira verde 

não foi entregue a caixa de correio de inox para muro e estou me sentindo lesada e também não recebi a nota fiscal com o valor pago a vista 

foi a primeira vez que fiz uma compra pela a internet e foi uma péssima esperiencia agora quero saber se vou ficar no prejuízo não recebi o produto e quero meu dinheiro de volta 

não recomendo até o momento não recebi o pr

gostaria de saber quando chegará meu produto pois já se passou a data prevista para entrega gostaria de uma resposta estou aguardando 

bom dia ao abrir a caixa verifiquei que o pedido brinco de gancho pendurado chapeado de prata cristal orelha prata 5 estava faltando e estava sendo especificado na nota fiscal 

não gostei do produto matérial de péssima qualidade eu não recomendaria a ninguém pensei que era uma coisa mais vi outra no anúncio falava que era felpudo de felpudo não tem nada nunca mais compro 

gostaria de saber onde foi parar o meu pedido de entrega e a solução do meu problema contrate firma de entrega competente 

não foi entregue na minha residencia tive que buscar nos correios 

boa noite efetuei uma compra de duas araras móvel paguei o valor de duas araras mais um frete embora eu tenha recebido no prazo não recebi o produto correto veio apenas uma peça 

não foi entregue no end tive q retirar nos correios 

não sei ao certo se o problema esta no fornecedor ou no trans

boa tarde venho informar que não recebi o produto e está constando no site que recebi gostaria de saber o que será feito por vocês sobre a minha encomenda 

boa noite comprei 3 produtos e recebi apenas 1 estou aguardando um retorno ref aos demais produtos comprados aguardo retorno breve 

não sei onde o produto se perdeu mas saiu de curitiba e não chegou no rio nenhuma satisfação 

foi a segunda compra com esse parceiro de vocês sendo que a primeira foi cancelada por conta que não tinha o produto e essa agora o produto ainda não chegou e já se passou do prazo de entrega 

por favor preciso receber esse produto com urgência espero que isso se resolva mais rápido possível 

boa noite o prazo se a entrega seria dia 29 03 18 hoje dia 03 04 ainda não recebi 

não recomendo o produto veio rasgado não corresponde à qualidade do produto do site se pudesse gostaria do dinheiro de volta 

não sei se o problema foi a transportadora ou a empresa de cadeiras mas das 3 cadeiras presidência são uma f

bom dia quero saber quando vou receber o meu pedido que já passou do prazo era para ser entregue no dia 21 03 e até agora não recebi todas as compras que faço nas lojas lannister não chegam na data 

boa noite não recebi o produto correios nunca atualizou o código de rastreio e nem uma posição da loja até o momento 

não foi entregue 

superou as expectativas 

so foi entregue o sexto os quadros não 

por favor eu já registrei há 48 horas atrás que não recebi a capa para colchão anti ácaro quemm da allergic center feita neste pedido recebi apenas as capas de travesseiro obs tel 9 8487 0788 

não gostei comprei 2 produtos no mesmo pedido e só recebi 1 pra entrar no site pra resolver o problema foi o maior caus com muito trabalho conseguir enviar 2 mensagens quando fui tentar não consigo 

realizei a compra de 2 colchas de solteiro e só recebi 1 produto e até agora não recebi a outra colcha 

não gostei do produto quero trocar em outro produto meu cabelo ficou muito ruim com esta escova 

In [150]:
y = e_set_rev_1.loc[e_set_rev_1['nr_tk_negativos'] > 1]
y.head()

,order_products_value,review_score,review_comment_message,review_tratada,nr_tk_positivos,porcent_tk_positivos,nr_tk_negativos,porcent_tk_negativos


In [162]:
e_set_rev_1.head(696)

,order_products_value,review_score,review_comment_message,review_tratada,nr_tk_positivos,porcent_tk_positivos,nr_tk_negativos,porcent_tk_negativos
0,134.90,1,o correio ainda não fez a entrega to furioso,correar ainda fazer entregar to furioso,0.0,0.000000,0.0,0.000000
1,329.99,1,não recebi o meu produto e nem fui avisada da ...,receber produto ser avisar falto item loja ent...,0.0,0.000000,0.0,0.000000
2,251.90,1,comprei um produto e recebi outro e inferior g...,comprar produto receber outro inferior gostar ...,0.0,0.000000,0.0,0.000000
3,979.00,1,não comprem só vai dar dor de cabeça o produto...,comprar ir dar dor cabeca produto nunca chegar,0.0,0.000000,0.0,0.000000
4,62.70,1,comprei 2 produtos iguais porém em cores difer...,comprar 2 produto igual pôr corar diferente ch...,0.0,0.000000,0.0,0.000000
5,975.00,1,ate o presente momento nao recebi meu produto ...,atar presentar momento receber produto achar u...,0.0,0.000000,0.0,0.000000
6,90.00,1,o produto não tem boa qualidade as tampas não ...,produto ter bom qualidade tampar fechar totalm...,0.0,0.000000,0.0,0.000000
7,70.74,1,até o momento nao recomento pois veio faltando...,atar momento recomento pois vir faltar produto...,0.0,0.000000,0.0,0.000000
8,129.00,1,é a segunda vez que compro o mesmo produto a p...,segundo vez comprar produto primeiro vez compr...,0.0,0.000000,0.0,0.000000
9,129.96,1,eu cancelei a compra estou esperando a devoluç...,cancelar comprar estar esperar devolucao valor...,0.0,0.000000,1.0,0.111111


In [26]:
vectorizer = CountVectorizer(encoding = 'utf_8')
reviews = []

for row in e_work_set.itertuples():
    reviews.append(row[3])

tokenized_reviews_all = vectorizer.fit_transform(reviews)

# A fazer

- Análise de Sentimento: Calcular porcentagem de palavras positivas, negativas e neutras em cada texto.
- Stemming

## Teste

In [163]:
a = 'eu não gostei do produto nao faltando'
nlp = pt_core_news_sm.load()
doc = nlp(a)
lista = [w.lemma_ for w in doc]
print(lista)
# print([(w.text, w.lemma_) for w in doc])


['eu', 'não', 'gostar', 'do', 'produto', 'nao', 'faltar']


In [116]:
print(' '.join(lematizar('ai meu deus que preguiça dessa lematizador'.split())))

ai meu deus que preguiçar d esse lematizador


10
